### Reflection pattern

Here we pass back the response with another request to critique the response.

This pattern can be useful in RAG workflows where we ask the LLM to assess the previous response.


In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display_markdown

In [2]:
client = OpenAI()
MODEL = "gpt-4o-mini"

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")


if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:10]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-Dg


We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python
programmer eager to receive feedback / critique by the user.


In [15]:
content = """
You are a Python programmer tasked with generating high quality Python code. Your task is to Generate the best content possible for the user's request. If the user provides critique,
respond with a revised version of your previous attempt.
"""

generation_chat_history = [{"role": "system", "content": content}]

Now, as the user, we are going to ask the LLM to generate an implementation of the Requests library


In [5]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of requesting an API with the Requests library",
    }
)

Let's generate the first version of the essay.


In [6]:
user_code = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)


generation_chat_history.append({"role": "assistant", "content": user_code})

In [7]:
display_markdown(user_code, raw=True)

Certainly! Below is a Python implementation that demonstrates how to make a request to an API using the `requests` library. This example includes handling different types of HTTP requests, setting headers, and handling responses.

```python
import requests

def fetch_data_from_api(url, params=None, headers=None):
    """
    Fetch data from the specified API using GET request.

    :param url: The API endpoint URL.
    :param params: Optional dictionary of query parameters.
    :param headers: Optional dictionary of HTTP headers.
    :return: JSON response from the API or None in case of an error.
    """
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()  # Raise an error for bad responses (4xx and 5xx)

        # Check if the response contains JSON data
        return response.json() if response.headers.get('Content-Type') == 'application/json' else response.text
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"Request exception occurred: {req_err}")
    except ValueError as json_err:
        print(f"JSON decoding error: {json_err}")

    return None

# Example usage
if __name__ == "__main__":
    api_url = "https://api.example.com/data"
    query_parameters = {'key': 'value'}
    headers = {'Authorization': 'Bearer YOUR_ACCESS_TOKEN'}

    result = fetch_data_from_api(api_url, params=query_parameters, headers=headers)
    if result:
        print("API Response:")
        print(result)
    else:
        print("Failed to retrieve data.")
```

### Explanation
- **Function Definition**: The function `fetch_data_from_api` is defined to accept a URL, optional query parameters, and optional headers.
- **GET Request**: It uses `requests.get` to make a GET request to the API.
- **Error Handling**: The function handles common exceptions like HTTP errors and request exceptions. 
- **Response Handling**: It checks if the response is JSON and returns it; otherwise, it returns the raw text.
- **Example Usage**: The script demonstrates how to call the function and handle the returned data.

Make sure to replace `"https://api.example.com/data"` and `'Bearer YOUR_ACCESS_TOKEN'` with the appropriate API endpoint and authentication token. You can also modify the query parameters based on the API requirements. 

Feel free to ask if you need further modifications or assistance!

## Reflection Step


This is equivalent to asking a followup question in say ChatGPT


In [8]:
reflection_chat_history = [
    {
        "role": "system",
        "content": "You are an experienced and talented Pythonista. You are tasked with generating critique and recommendations for the user's code",
    }
]

In [9]:
# We add new messages to the list of messages so that the LLM has context and knowledge of what proceeded.

# LLM calls are stateless and previous messages are not stored with the LLM. This is an important fact as we do not want to go over the context window for the LLM or incur unwanted costs if applicable.

reflection_chat_history.append({"role": "user", "content": user_code})

CRITIQUE TIME


Now that we have the context and the request for a critique, we make a request to the LLM.


In [10]:
critique = (
    client.chat.completions.create(messages=reflection_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [11]:
# Display

display_markdown(critique, raw=True)

Your implementation of making API requests using the `requests` library is solid and well-structured. Here are some critiques and recommendations for improvement:

### Critiques:

1. **Error Handling**: While you handle HTTP errors and request exceptions, it may be better to provide more feedback beyond just printing errors to the console. This can help users debug issues more effectively.

2. **Returning Non-JSON Responses**: If the API returns a non-JSON response, your current implementation returns `response.text` but does not clarify the context of that response. It would be beneficial to indicate whether the response was expected (like a success message) or might have been an error message.

3. **Documentation and Type Annotations**: While documentation strings are provided, adding type annotations to the function parameters and return type improves readability and usability, especially for those using type checkers or developing with IDEs that provide autocomplete features.

4. **Limiting Scope of `except` Statements**: Instead of catching all exceptions (`requests.exceptions.RequestException`), you might want to be specific with the exceptions you're expecting. Catching broad exceptions can hide unexpected errors.

5. **Global Constants**: If you have certain constants (like the API URL or the access token), consider defining them as constants at the beginning of the script instead of hard-coding them into the function or usage example.

### Recommendations:

1. **Enhanced Logging**: Use the `logging` module instead of `print` statements for better control over logging levels and output formatting. This way, you can easily toggle verbosity and redirect logs to files or other handlers.

   ```python
   import logging
   logging.basicConfig(level=logging.INFO)
   logger = logging.getLogger(__name__)
   ```

2. **Generic Request Function**: Consider extending your function to handle different types of HTTP requests (POST, PUT, DELETE) by allowing a method parameter.

   ```python
   def fetch_data_from_api(method='GET', url, params=None, headers=None, data=None):
       ...
       response = requests.request(method, url, params=params, headers=headers, json=data)
       ...
   ```

3. **Improved API Response Handling**: Clearly define what to do if the response is JSON, but it has a status indicating an error (e.g., 400 or 500). You might want to return both the status code and response data for better debugging.

4. **Optional Retry Logic**: For robustness, especially in network operations, consider implementing retry logic for transient issues, using `requests.adapters.HTTPAdapter`.

5. **Environment Variables for Sensitive Data**: Use environment variables to store sensitive information like the API URL and access tokens. This can prevent accidental exposure of credentials.

   ```python
   import os
   api_url = os.getenv("API_URL")
   headers = {'Authorization': f'Bearer {os.getenv("API_TOKEN")}'}
   ```

### Revised Example:

Here is how you could integrate some of these suggestions:

```python
import os
import requests
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def fetch_data_from_api(method='GET', url=None, params=None, headers=None, data=None):
    if url is None:
        logger.error("URL is required for the API request.")
        return None

    try:
        response = requests.request(method, url, params=params, headers=headers, json=data)
        response.raise_for_status()  # Raise an error for bad responses

        logger.info("Request succeeded: %s", response.status_code)
        return response.json() if response.headers.get('Content-Type') == 'application/json' else response.text
    except requests.exceptions.HTTPError as http_err:
        logger.error("HTTP error occurred: %s", http_err)
    except requests.exceptions.RequestException as req_err:
        logger.error("Request exception occurred: %s", req_err)

    return None

if __name__ == "__main__":
    api_url = os.getenv("API_URL", "https://api.example.com/data")
    query_parameters = {'key': 'value'}
    headers = {'Authorization': f'Bearer {os.getenv("API_TOKEN", "YOUR_ACCESS_TOKEN")}'}

    result = fetch_data_from_api(url=api_url, params=query_parameters, headers=headers)
    if result:
        print("API Response:", result)
    else:
        print("Failed to retrieve data.")
```

These changes should enhance the code's clarity, maintainability, and robustness. Let me know if you have further questions or need additional modifications!

Add CRITIQUE to chat...


In [12]:
generation_chat_history.append({"role": "user", "content": critique})

Response to CRITIQUE


In [13]:
essay = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [14]:
# Diaply user response to CRITIQUE
display_markdown(essay, raw=True)

Thank you for your detailed critique and suggestions! Here’s a revised implementation that incorporates your recommendations for improving the API request handling:

```python
import os
import requests
import logging
from typing import Optional, Dict, Any, Union

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def fetch_data_from_api(
    method: str = 'GET', 
    url: Optional[str] = None, 
    params: Optional[Dict[str, Any]] = None, 
    headers: Optional[Dict[str, str]] = None, 
    data: Optional[Any] = None
) -> Optional[Union[Dict[str, Any], str]]:
    """
    Fetch data from the specified API using the given HTTP method.

    :param method: HTTP method (GET, POST, etc.)
    :param url: The API endpoint URL.
    :param params: Optional dictionary of query parameters.
    :param headers: Optional dictionary of HTTP headers.
    :param data: Optional data to send with the request.
    :return: JSON response from the API, raw text if not JSON, or None in case of an error.
    """
    if url is None:
        logger.error("URL is required for the API request.")
        return None

    try:
        response = requests.request(method, url, params=params, headers=headers, json=data)
        response.raise_for_status()  # Raise an error for bad responses

        logger.info("Request succeeded: %s %s", response.status_code, response.url)
        # Check if the response contains JSON data
        if response.headers.get('Content-Type') == 'application/json':
            return response.json()
        else:
            logger.warning("Received non-JSON response.")
            return response.text
    except requests.exceptions.HTTPError as http_err:
        logger.error("HTTP error occurred: %s", http_err)
    except requests.exceptions.RequestException as req_err:
        logger.error("Request exception occurred: %s", req_err)

    return None

if __name__ == "__main__":
    # Retrieve API URL and token from environment variables
    api_url = os.getenv("API_URL", "https://api.example.com/data")
    query_parameters = {'key': 'value'}
    headers = {'Authorization': f'Bearer {os.getenv("API_TOKEN", "YOUR_ACCESS_TOKEN")}'}

    # Make the API request
    result = fetch_data_from_api(url=api_url, params=query_parameters, headers=headers)
    if result:
        print("API Response:", result)
    else:
        print("Failed to retrieve data.")
```

### Key Improvements:
1. **Enhanced Logging**: Logging is now more structured and informative, which will aid in troubleshooting issues if they arise.
2. **Dynamic HTTP Method**: The function supports multiple HTTP methods, making it reusable for various types of API interactions.
3. **Type Annotations**: Added type hints for clearer parameter expectations and return types, enhancing code readability and compatibility with IDEs.
4. **Environment Variables**: The API URL and token are retrieved from environment variables for security, preventing accidental exposure of sensitive information.
5. **Response Handling**: After making the request, the function provides feedback on whether the response was JSON or not, improving transparency.

This version should meet your expectations for a robust, clear, and maintainable API interaction implementation in Python. Let me know if you have any further questions or suggestions for improvement!

<br>

## We can of course make this a Class...
